In [1]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import Imputer

In [2]:
train = pd.read_csv('../Data/training2018.csv', header=0, index_col=0)

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
2016-11-08T07:55:00Z,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1818.0,920.0,False
2016-11-08T07:56:00Z,10.1,0.18,8.41,762.0,190.0,0.022,0.106,1805.0,927.0,False
2016-11-08T07:57:00Z,10.1,0.19,8.41,762.0,189.0,0.022,0.108,1787.0,927.0,False
2016-11-08T07:58:00Z,10.1,0.18,8.41,762.0,190.0,0.022,0.108,1790.0,936.0,False
2016-11-08T07:59:00Z,10.1,0.18,8.41,762.0,189.0,0.022,0.108,1827.0,924.0,False


In [ ]:
train.reset_index(inplace=True)
train.Time = pd.to_datetime(train.Time)

In [ ]:
null_df = train[train.isnull().any(axis=1)]

In [ ]:
null_df.count()

In [ ]:
print(null_df.shape)
print(train.columns)

In [ ]:
null_groups = null_df.groupby(null_df.Time.dt.date)

In [ ]:
null_groups.describe()

In [ ]:
grouped = train.groupby(train.Time.dt.date)

In [ ]:
null_groups.head()

In [ ]:
#grouped.get_group('2016-10-30').isnull().sum()
#grouped.get_group('2016-08-30').count()
#grouped.get_group('2016-08-30').isnull().sum()
grouped.get_group('2016-08-29').count()
#grouped.get_group('2016-08-29').isnull().sum()

In [ ]:
train.groupby(train.Time.dt.date).describe().head()

In [ ]:
import pdb
for day in null_groups.groups.keys():
    #pdb.set_trace()
    columns = train.columns.values.tolist()
    not_null_columns = train.groupby(train.Time.dt.date).get_group(str(day)).dropna(axis=1).columns.values.tolist()
    columns_to_be_filled = set(columns) - set(not_null_columns)
    #columns_to_be_filled = map(lambda x: x, columns_to_be_filled)
   
    means = train.groupby(train.Time.dt.date).get_group(str(day)).mean()
    means = means[columns_to_be_filled]
    idxs = null_groups.get_group(str(day)).index.values.tolist()
    train.loc[idxs, columns_to_be_filled] = means.tolist()

In [ ]:
s =null_groups = null_df.groupby(null_df.Time.dt.date).get_group('2016-08-30')
s

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

## Test set imputation

In [206]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import Imputer

In [207]:
test = pd.read_csv('../Data/testing2018.csv', header=0)
test.head()

,Time,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
0,2016-11-08T07:55:00Z,10.1,0.17,8.41,762.0,190.0,0.022,0.106,1818.0,920.0,False
1,2016-11-08T07:56:00Z,10.1,0.18,8.41,762.0,190.0,0.022,0.106,1805.0,927.0,False
2,2016-11-08T07:57:00Z,10.1,0.19,8.41,762.0,189.0,0.022,0.108,1787.0,927.0,False
3,2016-11-08T07:58:00Z,10.1,0.18,8.41,762.0,190.0,0.022,0.108,1790.0,936.0,False
4,2016-11-08T07:59:00Z,10.1,0.18,8.41,762.0,189.0,0.022,0.108,1827.0,924.0,False


In [208]:
#test.reset_index(inplace=True)
test.Time = pd.to_datetime(test.Time)

In [209]:
test.shape

(139566, 11)

In [210]:
null_df = test[test.isnull().values.any(axis=1)]

In [211]:
null_df.shape

(24480, 11)

In [212]:
null_groups = null_df.groupby(null_df.Time.dt.date)

In [213]:
for day in null_groups.groups.keys():
    columns = test.columns.values.tolist()
    #not_null_columns = test.groupby(test.Time.dt.date).get_group(str(day)).dropna(axis=1).columns.values.tolist()
    not_null_columns = [x for x in test.isnull().any().index if test.isnull().any()[x]==False]
    columns_to_be_filled = set(columns) - set(not_null_columns)
    columns_to_be_filled = list(columns_to_be_filled)
    means = test.groupby(test.Time.dt.date).get_group(str(day)).mean()
    means = means[columns_to_be_filled]
    idxs = null_groups.get_group(str(day)).index.values.tolist()
    test.loc[idxs, columns_to_be_filled] = means.tolist()

In [214]:
test[test.isnull().values.any(axis=1)].shape

(21600, 11)

In [215]:
test = test.dropna()

In [216]:
test.shape

(117966, 11)

In [219]:
test.to_csv('../Data/testing2018-mean_imputed.csv', index=False)